<a href="https://colab.research.google.com/github/SakshiPunia314/Sentiment-Analysis-Project-The-Sleep-Company/blob/main/sleepwell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries (run only if needed)
!pip install transformers
!pip install pandas

# Import libraries
import pandas as pd
from transformers import pipeline

# Load your scraped reviews
df = pd.read_csv('reviews.csv')

# Load a pre-trained sentiment analysis model
# We are using DistilBERT fine-tuned for sentiment
sentiment_pipeline = pipeline('sentiment-analysis')

# Function to classify sentiment
def get_sentiment(text):
    try:
        result = sentiment_pipeline(text[:512])[0]  # Limit text to 512 tokens
        if result['label'] == 'POSITIVE':
            return 'Positive'
        elif result['label'] == 'NEGATIVE':
            return 'Negative'
        else:
            return 'Neutral'
    except:
        return 'Neutral'

# Apply sentiment analysis
df['sentiment_label'] = df['review_text'].apply(get_sentiment)

# Check first few rows
df.head()

# Save the updated CSV
df.to_csv('reviews_with_sentiment.csv', index=False)

print("✅ Sentiment labeling done! Saved as 'reviews_with_sentiment.csv'.")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


✅ Sentiment labeling done! Saved as 'reviews_with_sentiment.csv'.


In [ ]:
# Install pandas if not already installed
!pip install pandas

# Import pandas
import pandas as pd

# Load the labeled reviews file
df = pd.read_csv('reviews_with_sentiment.csv')

# Clean star_rating column (some star ratings are like "5 stars", we need to extract the number)
df['star_rating'] = df['star_rating'].str.extract('(\d+)').astype(float)

# Group by store
grouped = df.groupby('store_name')

# Calculate metrics
metrics = grouped.agg(
    total_reviews=('review_text', 'count'),
    avg_star_rating=('star_rating', 'mean'),
    positive_reviews=('sentiment_label', lambda x: (x == 'Positive').sum()),
    negative_reviews=('sentiment_label', lambda x: (x == 'Negative').sum()),
    neutral_reviews=('sentiment_label', lambda x: (x == 'Neutral').sum())
)

# Calculate percentages
metrics['% Positive'] = (metrics['positive_reviews'] / metrics['total_reviews']) * 100
metrics['% Negative'] = (metrics['negative_reviews'] / metrics['total_reviews']) * 100
metrics['% Neutral'] = (metrics['neutral_reviews'] / metrics['total_reviews']) * 100

# Reset index to make 'store_name' a column
metrics = metrics.reset_index()

# Show the final table
metrics.head()

# Save to a new CSV
metrics.to_csv('store_sentiment_metrics.csv', index=False)

print("✅ Metrics calculation done! Saved as 'store_sentiment_metrics.csv'.")


✅ Metrics calculation done! Saved as 'store_sentiment_metrics.csv'.


In [ ]:
# Assuming you have already loaded 'reviews_with_sentiment.csv' earlier
import pandas as pd

# Load your labeled review data
df = pd.read_csv('reviews_with_sentiment.csv')

# Define target entities (keywords)
entities = ["mattress", "delivery", "service"]

# Create a blank list to store results
entity_sentiments = []

# Check each review
for index, row in df.iterrows():
    review_text = str(row['review_text']).lower()  # convert to lowercase
    for entity in entities:
        if entity in review_text:
            entity_sentiments.append({
                'entity': entity,
                'store_name': row['store_name'],
                'review_text': row['review_text'],
                'sentiment_label': row['sentiment_label']
            })

# Create a new dataframe for entity-level sentiments
entity_df = pd.DataFrame(entity_sentiments)

# Save it if you want
entity_df.to_csv('entity_level_sentiment.csv', index=False)

print("✅ Entity-level sentiment analysis done! Saved as 'entity_level_sentiment.csv'.")


✅ Entity-level sentiment analysis done! Saved as 'entity_level_sentiment.csv'.
